# Scraping Alcampo

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import time

In [2]:
HEADER = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) "
                        "Chrome/90.0.4430.85 Safari/537.36"}

In [30]:
alcampo_url_dict = {101: f'https://www.alcampo.es/compra-online/frescos/verduras-y-hortalizas/verduras-ecologicas/hortaliza-ecologica-tomate-de-ensalada-ecologico-600g/p/55576',
                102: f'https://www.alcampo.es/compra-online/alimentacion/ecologicos/productos-frescos-ecologicos/frutas-y-verduras-ecologicas/frutas-ecologicas/fruta-ecologica-platanos-ecologicos-bandeja-de-800-g/p/57540',
                103: f'https://www.alcampo.es/compra-online/frescos/frutas/frutas-ecologicas/fruta-manzanas-royal-ecologica-800-g/p/57561',
                104: f'https://www.alcampo.es/compra-online/frescos/verduras-y-hortalizas/verduras-ecologicas/hortaliza-ecologica-calabacin-ecologico-600-g/p/55542', 
                105: f'https://www.alcampo.es/compra-online/frescos/verduras-y-hortalizas/verduras-ecologicas/alcampo-produccion-controlada-ecologico-zanahoria-ecologico-700-g/p/58394',
                106: f'https://www.alcampo.es/compra-online/frescos/verduras-y-hortalizas/verduras-ecologicas/alcampo-produccion-controlada-ecologico-patata-ecologica-malla-de-2-kg/p/58396'}
                #302: f'https://www.alcampo.es/compra-online/alimentacion/desayuno-y-merienda/bolleria-y-pasteleria/croissants-magdalenas-y-muffins/magdalenas/la-granja-mini-magdalenas-ecologicas-chocochip-con-yogur-200-g/p/563256',
                #303: f'https://www.alcampo.es/compra-online/alimentacion/desayuno-y-merienda/bolleria-y-pasteleria/rosquillas-y-sobaos/rosquillas/monjas-rosquillas-250-g/p/13606',
                #304: f'https://www.alcampo.es/compra-online/alimentacion/alimentos-especiales-/alimentos-especiales/dulces-y-desayunos/bolleria/la-granja-bizcocho-marmol-ecologico-300-g/p/801212', 
                #702: f'https://www.alcampo.es/compra-online/alimentacion/alimentos-especiales-/alimentos-especiales/bebidas-soja-avena-y-arroz/bebidas-de-avena/soria-natural-bebida-de-avena-1-l/p/12207',
                #704: f'https://www.alcampo.es/compra-online/alimentacion/leche-huevos-yogures-y-lacteos/yogures-bifidus-y-lcasei/desnatado/sabores/casa-grande-de-xanceda-yogur-de-fresa-ecologico-2-uds-x-125-g/p/52971',
                #705: f'https://www.alcampo.es/compra-online/alimentacion/ecologicos/productos-frescos-ecologicos/lacteos-quesos-y-huevos-ecologicos/leche-y-nata-ecologicas/puleva-eco-leche-semidesnatada-ecologica-1-l/p/53042',
                #707: f'https://www.alcampo.es/compra-online/alimentacion/ecologicos/productos-frescos-ecologicos/lacteos-quesos-y-huevos-ecologicos/kefir-ecologico/cantero-de-letur-kefir-de-cabra-bio-420-gramos/p/519624',
                #1001: f'https://www.alcampo.es/compra-online/alimentacion/caldos-pasta-arroz-legumbres-pure/arroz/arroz-para-cocinar/calasparra-arroz-redondo-1-kg/p/25214',        
                #1101: f'https://www.alcampo.es/compra-online/alimentacion/desayuno-y-merienda/cafessucedaneos-y-derivados-del-cacao/cafe-molido/molido-natural/intermon-oxfam-tierra-madre-cafe-molido-natural-250-g/p/16348',
                #1201: f'https://www.alcampo.es/compra-online/bebidas/cervezas/lata-estandar/mahou-cerveza-33cl/p/30378'}

In [26]:
url_alcampo = f'https://www.alcampo.es/compra-online/frescos/verduras-y-hortalizas/verduras-ecologicas/hortaliza-ecologica-tomate-de-ensalada-ecologico-600g/p/55576'
html = requests.get(url_alcampo, headers=HEADER).text
soup_alcampo = BeautifulSoup(html, 'lxml')
price_alcampo = soup_alcampo.find('span', {'class': 'precioUnidad pesoVariable'}).text
precio = re.findall('[0-9,]+', price_alcampo)[0].replace(',',('.'))
unidad = re.findall('(\€.*)\)', price_alcampo)[0]

In [27]:
precio, unidad

('3.81', '€/Kg')

In [33]:
def updating_alcampo():
    
    #This function creates a tiny dataframe with the product's prices updated and dated every time it runs. Raws will be added to the main database.
    
    update_alcampo_dataframe = pd.DataFrame(columns=['id', 'producto', 'tienda', 'precio', 'unidad', 'fecha'])
    tienda = 'Alcampo'
    fecha = time.strftime('%A %H:%M %d-%m-%Y')
    
    # Temporary list of product to match id. TO DO: products table
    products_dict = {101: 'tomate ensalada', 102: 'plátano canario', 103: 'manzana royal', 104: 'calabacín', 105: 'zanahoria', 106: 'patata blanca'}
    
    print('Starting to scrape Alcampo...')
    
    for product_id, url in alcampo_url_dict.items():
        print(url)
        time.sleep(5)
        html = requests.get(url, headers=HEADER).text
        soup_alcampo = BeautifulSoup(html, 'lxml')
        
        try:
            price_alcampo = soup_alcampo.find('span', {'class': 'precioUnidad pesoVariable'}).text
        
        except:
            price_alcampo = soup_alcampo.find('span', {'class': 'pesoVariable precioUnidad'}).text
        
        if price_alcampo is None:
        
            precio = 'No disponible'
            unidad = 'No disponible'
        
        else:
            precio = re.findall('[0-9,]+', price_alcampo)[0].replace(',',('.'))
            unidad = re.findall('(\€.*)\)', price_alcampo)[0]

        print(product_id, '--', precio, '--', unidad)

        producto = [v for k, v in products_dict.items() if k == product_id]
        
        # Adding new line to the dataframe
        update_alcampo_dataframe.loc[len(update_alcampo_dataframe)] = [product_id, producto[0], tienda, precio, unidad, fecha]
    
    return update_alcampo_dataframe

In [34]:
updating_alcampo()

Starting to scrape Alcampo...
https://www.alcampo.es/compra-online/frescos/verduras-y-hortalizas/verduras-ecologicas/hortaliza-ecologica-tomate-de-ensalada-ecologico-600g/p/55576
101 -- 3.81 -- €/Kg
https://www.alcampo.es/compra-online/alimentacion/ecologicos/productos-frescos-ecologicos/frutas-y-verduras-ecologicas/frutas-ecologicas/fruta-ecologica-platanos-ecologicos-bandeja-de-800-g/p/57540
102 -- 1.86 -- €/Kg
https://www.alcampo.es/compra-online/frescos/frutas/frutas-ecologicas/fruta-manzanas-royal-ecologica-800-g/p/57561
103 -- 4.36 -- €/Kg
https://www.alcampo.es/compra-online/frescos/verduras-y-hortalizas/verduras-ecologicas/hortaliza-ecologica-calabacin-ecologico-600-g/p/55542
104 -- 2.81 -- €/Kg
https://www.alcampo.es/compra-online/frescos/verduras-y-hortalizas/verduras-ecologicas/alcampo-produccion-controlada-ecologico-zanahoria-ecologico-700-g/p/58394
105 -- 2.27 -- €/Kg
https://www.alcampo.es/compra-online/frescos/verduras-y-hortalizas/verduras-ecologicas/alcampo-produccion-

,id,producto,tienda,precio,unidad,fecha
0,101,tomate ensalada,Alcampo,3.81,€/Kg,Tuesday 19:13 08-06-2021
1,102,plátano canario,Alcampo,1.86,€/Kg,Tuesday 19:13 08-06-2021
2,103,manzana royal,Alcampo,4.36,€/Kg,Tuesday 19:13 08-06-2021
3,104,calabacín,Alcampo,2.81,€/Kg,Tuesday 19:13 08-06-2021
4,105,zanahoria,Alcampo,2.27,€/Kg,Tuesday 19:13 08-06-2021
5,106,patata blanca,Alcampo,1.32,€/Kg,Tuesday 19:13 08-06-2021
